## Adding Party Affiliation to Speakers

Data for this research was acquired from the outstanding work of Gentzkow, Shapiro and Taddy (2018) (data can be found <a href='https://data.stanford.edu/congress_text'>here</a>). Political party affiliation for each speaker is not provided in this original data. To associate party with representatives the <i>ProPublica Members API</i> is used. This process is not 100% accurate due to text-parsing and transcription issues in the original dataset. However, through both an automated and manual process, over 95% of speeches were given accurate party labels for each year in the current corpus (1983-2016).

This notebook was run manually for each congress. As such, the current state of this notebook represents the last congress to be processed (manual editing and printed values thus differ between congresses).

In [3]:
import numpy as np
import pandas as pd

import boto3
client = boto3.client('s3')

import re
import unidecode

import requests
import json
from io import StringIO

%run us_state_abbrev.py

### Read in originally parsed data

In [5]:
# Data from S3 bucket
DF = pd.read_csv(client.get_object(Bucket='ascsagemaker',
                                       Key=f'JMP_congressional_nmf/House_bigrams/098.csv')['Body'])
DF = DF.loc[DF.chamber_x == 'H']

# get the member info for the given congress
rq = requests.get('https://api.propublica.org/congress/v1/98/house/members.json',
                  headers={"X-API-Key":'WDmcjspeHVFMDSmrIZ3NV2gKAESs5vldAS8rz3X6'})

ref = pd.DataFrame(json.loads(rq.text)['results'][0]['members'])

# format information to match with names in corpus
ref = ref[['first_name','last_name','gender','state','party']]
ref['state_long'] = ref.state.apply(lambda x: abbrev_to_us_state[x].lower())
ref['last_name'] = ref['last_name'].apply(lambda x: unidecode.unidecode(x.lower()))

### How it will work

The congressional record includes the name of the representative speaking before the speech. Ordinarily the honorifics of the speaker are written before the last name of the speaker. However, in the case that the individual has a last name and gender that is shared with another representative, that representatives state is also entered (e.g. Mr. Young of Alaska). 

To match data to the propublica database, the speaker name must be parsed to extract these three pieces of information (honorific/gender, last name, state).

In [1085]:
# get all unique names from corpus
unique_names = DF.speaker.unique()

match_df = []
unmatched = []

for name in tqdm(unique_names):
    # normalize text
    norm = name.lower()
    
    # filter by gender first
    if re.findall('mr\..*',norm):
        gender_ref = ref.loc[ref.gender == 'M']
    elif re.findall('mrs\..*',norm) or re.findall('ms\..*',norm):
        gender_ref = ref.loc[ref.gender == 'F']
    else:
        gender_ref = ref
    
    # find name matches
    rows = []
    for ix,row in gender_ref.iterrows():
        if re.findall(r'\b%s\b'%row.last_name,norm):
            rows.append(row)
    rows = pd.DataFrame(rows)
    
    if len(rows) > 0:
        if len(rows) == 1:  # if there is only one match, done
            rows['speaker'] = name
            match_df.append(rows)
        else:
            # if more than one match, look to state information
            split_name = norm.split('of')
            
            if len(split_name) > 1:
                state = split_name[1].strip()
                rows = rows.loc[rows.state_long.str.contains(state)]
                
                if len(rows) == 1:  # if there is only one match, done
                    rows['speaker'] = name
                    match_df.append(rows)
                
                # if still more than one name match in the same state, look for first name
                elif len(rows) > 1:
                    n_rows = []
                    for ix,row in rows.iterrows():
                        if re.findall(r'\b%s\b'%row['first_name'].lower(),norm):
                            n_rows.append(row)
                    if len(n_rows) != 1:
                        unmatched.append(name)
                    else:  #  add final name match
                        n_rows = pd.DataFrame(n_rows)
                        n_rows['speaker'] = name
                        match_df.append(n_rows)
            else:
                unmatched.append(name)

#  some diagnostics
match_df = pd.concat(match_df)
match_df = match_df[['speaker','party']]
df2 = DF.merge(match_df,on='speaker',how='outer')
print(f"original DF length was - {len(DF)}")
print(f'removed {len(DF) - (len(df2.loc[-df2.party_y.isnull()]))} or {(len(DF) - (len(df2.loc[-df2.party_y.isnull()])))/(len(DF)):2f}')

100%|██████████| 805/805 [00:23<00:00, 34.80it/s]


original DF length was - 40363
removed 7282 or 0.180413


### For those names that were not matched automatically and have a substantial number of speeches, match them manually

In [1086]:
for v,i in df2.loc[df2.party_y.isnull()].groupby('speaker').count().reset_index()[['speaker','date']].iterrows():
    if i['date'] > 20:
        print(i)

speaker    Mr. ANDERSON
date                 95
Name: 6, dtype: object
speaker    Mr. AuCOIN
date              210
Name: 9, dtype: object
speaker    Mr. BATES
date              36
Name: 11, dtype: object
speaker    Mr. BEDELL
date              192
Name: 14, dtype: object
speaker    Mr. BIAGGI
date              278
Name: 17, dtype: object
speaker    Mr. BONKER
date              179
Name: 18, dtype: object
speaker    Mr. CHANDLER
date                 50
Name: 31, dtype: object
speaker    Mr. CONABLE
date               195
Name: 37, dtype: object
speaker    Mr. CONTE
date             378
Name: 38, dtype: object
speaker    Mr. DENNY SMITH
date                    44
Name: 48, dtype: object
speaker    Mr. DONNELLY
date                 65
Name: 52, dtype: object
speaker    Mr. DYMALLY
date                40
Name: 58, dtype: object
speaker    Mr. DYSON
date              41
Name: 59, dtype: object
speaker    Mr. ECKART
date               65
Name: 61, dtype: object
speaker    Mr. EDWARDS of Okla

In [1104]:
ref.loc[ref.last_name == 'won pat']

,first_name,last_name,gender,state,party,state_long
436,Antonio,won pat,None,GU,D,guam


In [1105]:
df2.loc[df2.speaker == 'Mr. MORRISON of Connecticut','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. MORRISON of Washington','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. ROBERT F. SMITH','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. ROBINSON','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. ST GERMAIN','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. FOWLER','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. HANCE','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. LEVITAS','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. WEISS','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. WON PAT','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. PATTERSON','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. PERKINS','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. SAM B. HALL. JR','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. HIGHTOWER','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. PHILIP M. CRANE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. RUDD','party_y'] = 'R'

df2.loc[df2.speaker == 'Mr. FRENZEL','party_y'] = 'R'

df2.loc[df2.speaker == 'Mr. SIKORSKI','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. SNYDER','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. CONABLE','party_y'] = 'R'

df2.loc[df2.speaker == 'Mr. STANGELAND','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. McKINNEY','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. RUDD','party_y'] = 'R'

df2.loc[df2.speaker == 'Mr. MICHEL','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. MITCHELL','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. SUNIA','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. JONES of Tennessee','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. TAYLOR','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. HILLIS','party_y'] = 'R'

df2.loc[df2.speaker == 'Mr. WALGREN','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BEDELL','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. FUQUA','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. UDALL','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. WALGREN','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BONKER','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. ANDERSON','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BIAGGI','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. AuCOIN','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BATES','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. FLIPPO','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BOSCO','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BRUCE','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. BOULTER','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. LATTA','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. CALLAHAN','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. CONTE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. CHANDLER','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. DENNY SMITH','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. SMITH of Mississippi','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. DYMALLY','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. DYSON','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. DONNELLY','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. ECKART','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. ECKART of Ohio','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. ERDREICH','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. PAYNE of Virginia','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. FASCELL','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. THOMAS A. LUKEN','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. GREEN','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. GREEN of New York','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. HERTELL','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. HERTEL of Michigan','party_y'] = 'D'

df2.loc[df2.speaker == 'Mr. HERTEL','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. HOLLOWAY','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. HUBBARD','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. JAMES','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. LENT','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. LEVINE of California','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. MARLENEE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. MOODY','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. PARRIS','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. McEWEN','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. HOPKINS','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. STANGELAND','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. SAVAGE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. SCHUETTE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. SHUSTER','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. SHUMWAY','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. TRAXLER','party_y'] = 'D'
df2.loc[df2.speaker == 'Mr. WYLIE','party_y'] = 'R'
df2.loc[df2.speaker == 'Mr. YATRON','party_y'] = 'D'
df2.loc[df2.speaker == 'Ms. LONG','party_y'] = 'R'

In [1106]:
print(f'removed {len(DF) - (len(df2.loc[-df2.party_y.isnull()]))} or {(len(DF) - (len(df2.loc[-df2.party_y.isnull()])))/(len(DF)):2f}')

removed 769 or 0.019052


### Save out to S3

In [1107]:
csv_buffer = StringIO()
df2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('ascsagemaker', 'JMP_congressional_nmf/House_bigrams/098_fixed_party.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'JEB9RXA7SKNJRQ4H',
  'HostId': 'pJpXXtLM2EbEgOAKgWSTsULJUpcWHcJZhD0vnoM9xqFnPB+amfS+PXxOR2w3ed8rXj/9lfBKBKo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pJpXXtLM2EbEgOAKgWSTsULJUpcWHcJZhD0vnoM9xqFnPB+amfS+PXxOR2w3ed8rXj/9lfBKBKo=',
   'x-amz-request-id': 'JEB9RXA7SKNJRQ4H',
   'date': 'Tue, 04 Jan 2022 16:32:22 GMT',
   'x-amz-version-id': '.MEnkWzDAzvhKfE_5ie8VsxgL1SQwRn7',
   'etag': '"15051d1093da73f43f220b28f5563287"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"15051d1093da73f43f220b28f5563287"',
 'VersionId': '.MEnkWzDAzvhKfE_5ie8VsxgL1SQwRn7'}